In [0]:
#imports:
import pandas as pd
import numpy as np
import math
from time import time

from sklearn.utils import shuffle

import xgboost

from scipy.stats import randint as sp_randint # int distribution for random search
from scipy.stats import uniform # float distribution for random search
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from matplotlib import pyplot as plt

In [2]:
from google.colab import drive 
drive.mount('/mntDrive') 
data=pd.read_csv("/mntDrive/My Drive/НИР магистратура/Оптимизация гиперпараметров алгоритмов градиентного бустинга/Практика/data/input/prepared_facebook_data.csv")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /mntDrive


In [3]:
# grid search
# get some data
X, y = data.iloc[:,:-1].values, data.iloc[:,-1].values

# build a classifier
clf = xgboost.XGBRegressor(tree_method = "gpu_hist", gpu_id=0, verbosity=0)

# use a full grid over 4 parameters
param_grid = {'max_depth': [2, 5, 8], 'min_child_weight': [2, 5, 8], 
              'alpha':[0.25,0.5,0.75], 'lambda':[0.25,0.5,0.75],
              'subsample':[0.625, 0.75, 0.875],'colsample_bytree':[0.625, 0.75, 0.875]} 

# run grid search
grid_search = GridSearchCV(clf, param_grid=param_grid, cv=5, scoring="r2" ,iid=False)
start = time()
grid_search.fit(X, y)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))

res_grid=pd.DataFrame(grid_search.cv_results_)
res_grid['experiment_name']='grid search'

GridSearchCV took 1940.05 seconds for 729 candidate parameter settings.


In [4]:
# run with default values
default = GridSearchCV(clf, param_grid={'max_depth': [3]}, cv=5, scoring="r2", iid=False)#default values
start = time()
default.fit(X, y)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(default.cv_results_['params'])))

default=pd.DataFrame(default.cv_results_)
default['experiment_name']='default params'

GridSearchCV took 2.02 seconds for 1 candidate parameter settings.


In [0]:
res= pd.concat([res_grid, default], sort=False)

In [0]:
res.to_csv("res_grid_GPU.csv",index=False)

In [0]:
cp res_grid_GPU.csv "/mntDrive/My Drive/НИР магистратура/Оптимизация гиперпараметров алгоритмов градиентного бустинга/Практика/data/results/xgboost_perfomance_grid_search_GPU.csv"